In [1]:
import sys
sys.path.append('./gen-py')
from pydata.ttypes import *
from thrift.protocol import TBinaryProtocol
from thrift.transport import TTransport
import pandas as pd
from IPython.display import Image
import pickle

In [ ]:
item = WikiLinkItem()
for filenum in range(1, 110):
    dataList = []
    filename = '%03d' % filenum
    dataFile = open(filename, 'rb')
    proto = TBinaryProtocol.TBinaryProtocol(TTransport.TBufferedTransport(dataFile))
    while True:
        try:
            item.read(proto)
        except EOFError:
            break
        for mention in item.mentions:
            if mention.context is not None:
                row = (mention.anchor_text, (mention.context.left + ' ' + mention.context.right), mention.wiki_url)
                dataList.append(row)
    pickle.dump(dataList, open('allMentions' + filename + '.pkl', 'wb'))

In [9]:
from customTransformers import SelectNameTransformer
import pickle

name = 'Susan'  # The name to be selected
snt = SelectNameTransformer(name)

nameData = []
for filenum in range(1, 110):
    filename = '%03d' % filenum
    mentions = pickle.load(open('raw/allMentions' + filename + '.pkl', 'rb'))
    mentions = snt.transform(mentions)
    nameData += mentions

context = [mention[0] for mention in nameData]
link = [mention[1] for mention in nameData]

pickle.dump(context, open('processed/context_' + name.lower() + '.pkl', 'wb'))
pickle.dump(link, open('processed/link_' + name.lower() + '.pkl', 'wb'))

!touch .done

In [2]:
dataList = []
for filenum in range(80, 110):
    filename = '%03d' % filenum
    allMentions = pickle.load(open('allMentions' + filename + '.pkl', 'rb'))
    dataList += allMentions
pickle.dump(dataList, open('allMentions80-109.pkl', 'wb'))